In [377]:
import pandas as pd
import numpy as np

import gc
import os
import random
import time

import seaborn as sns
from matplotlib import pyplot as plt

import statistics as st
from scipy.stats import skew 
from scipy.stats import kurtosis
from scipy import signal

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder, Normalizer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier

from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV, train_test_split

import xgboost as xgb

In [378]:
SEED = 1729
def determinism(SEED=SEED):
    random.seed = SEED
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed = SEED
    print('done')
    
determinism()

done


In [379]:
gsr=pd.read_csv('/content/gsr_train.csv',header=None)
rr=pd.read_csv('/content/rr_train.csv',header=None)
hr=pd.read_csv('/content/hr_train.csv',header=None)
temp=pd.read_csv('/content/temp_train.csv',header=None)
label=pd.read_csv('/content/labels_train.csv',header=None)

In [324]:
gsr.iloc[:,:] = Normalizer(norm='l1').fit_transform(gsr)
hr.iloc[:,:] = Normalizer(norm='l1').fit_transform(hr)
rr.iloc[:,:] = Normalizer(norm='l1').fit_transform(rr)
temp.iloc[:,:] = Normalizer(norm='l1').fit_transform(temp)

Power Spectrum Calculation

In [380]:
gsr_ps=[]
for i in range(0,632):    
    freqs, psd = signal.welch(gsr.iloc[i])
    gsr_ps.append(np.mean(psd))
rr_ps=[]
for i in range(0,632):    
    freqs, psd = signal.welch(rr.iloc[i])
    rr_ps.append(np.mean(psd))
hr_ps=[]
for i in range(0,632):    
    freqs, psd = signal.welch(hr.iloc[i])
    hr_ps.append(np.mean(psd))

/usr/local/lib/python3.6/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 30, using nperseg = 30
  .format(nperseg, input_length))


In [381]:
gsr_mean=(gsr.mean(axis=1))
gsr_mean2_gsr=(gsr.mean(axis=1))**2
#gsr_var=(gsr.var(axis=1))
gsr_var=(gsr.std(axis=1))
gsr_med=gsr.median(axis=1)
gsr_kurt=gsr.kurt(axis=1)
gsr_cs=np.cumsum(gsr,axis=1)[29]
#gsr_ps
hr_mean=(hr.mean(axis=1))
hr_mean2=(hr.mean(axis=1))**2
#hr_var=(hr.var(axis=1))
hr_var=(hr.std(axis=1))
hr_med=hr.median(axis=1)
hr_kurt=hr.kurt(axis=1)
hr_cs=np.cumsum(hr,axis=1)[29]
#hr_ps

rr_mean=(rr.mean(axis=1))
rr_mean2=(rr.mean(axis=1))**2
#rr_var=(rr.var(axis=1))
rr_var=(rr.std(axis=1))
rr_med=rr.median(axis=1)
rr_kurt=rr.kurt(axis=1)
rr_cs=np.cumsum(rr,axis=1)[29]
#rr_ps

#temp_i = temp[0]

In [382]:
df=pd.DataFrame(
    list(
        zip(gsr_mean,gsr_var,gsr_med,gsr_kurt,gsr_cs,gsr_ps,
            hr_mean,hr_var,hr_med,hr_kurt,hr_cs,hr_ps,
            rr_mean,rr_var,rr_med,rr_kurt,rr_cs,rr_ps, 
            #temp_i,
            label[0])))



In [384]:
df.columns=['gsr_mean','gsr_var','gsr_med','gsr_kurt','gsr_cs','gsr_ps',
            'hr_mean','hr_var','hr_med','hr_kurt','hr_cs','hr_ps',
            'rr_mean','rr_var','rr_med','rr_kurt','rr_cs','rr_ps', 
            #'temp_initial',
            'label']


In [385]:
df

,gsr_mean,gsr_var,gsr_med,gsr_kurt,gsr_cs,gsr_ps,hr_mean,hr_var,hr_med,hr_kurt,hr_cs,hr_ps,rr_mean,rr_var,rr_med,rr_kurt,rr_cs,rr_ps,temp_initial,label
0,1.629602,0.023946,1.633575,-1.584146,48.888074,0.000654,62.511111,1.443000,62.000000,1.000413,1875.333333,0.723715,0.942426,0.040319,0.951275,1.669456,28.272768,0.000989,33.619999,0
1,0.574807,0.122610,0.655739,-0.966358,17.244212,0.030699,74.933333,2.409114,74.666667,-0.338040,2248.000000,13.899084,0.804159,0.062397,0.788120,-0.332033,24.124768,0.006959,34.290001,1
2,1.020229,0.032426,1.039323,0.185807,30.606870,0.000733,62.500000,1.335487,63.000000,4.560340,1875.000000,0.823004,0.923990,0.237954,0.956805,3.917334,27.719701,0.008351,32.040001,1
3,1.507063,0.052266,1.504966,-0.961365,45.211902,0.002291,75.466667,1.176396,75.666667,3.515310,2264.000000,1.438966,0.793835,0.031040,0.790885,-0.916222,23.815051,0.001574,35.009998,0
4,0.378422,0.002689,0.377606,-1.675707,11.352647,0.000014,73.155556,7.828494,76.166667,3.425780,2194.666667,19.293753,0.701381,0.133792,0.745257,2.406221,21.041421,0.008772,31.900000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,3.120945,0.025381,3.105660,-0.103505,93.628362,0.000703,84.777778,1.235904,84.333333,-0.977876,2543.333333,3.900179,0.698708,0.120967,0.681655,4.770574,20.961227,0.031894,31.760000,0
628,2.678209,0.007556,2.677381,-0.727722,80.346274,0.000126,74.555556,2.687182,75.000000,0.865552,2236.666667,16.161541,0.790332,0.065638,0.790885,0.386511,23.709968,0.006209,32.549999,1
629,1.228856,0.018503,1.228508,-1.158179,36.865673,0.000804,80.933333,1.818171,80.333333,4.022358,2428.000000,2.051780,0.730601,0.040050,0.724517,0.257702,21.918032,0.001117,32.880001,0
630,0.659290,0.003409,0.659197,-1.183209,19.778713,0.000005,72.077778,1.995557,72.000000,-0.156441,2162.333333,5.004645,0.821304,0.028967,0.824069,0.283480,24.639120,0.001536,34.270000,0


In [386]:
np.corrcoef(hr_ps,gsr_ps)

array([[ 1.        , -0.03569933],
       [-0.03569933,  1.        ]])

In [387]:
cv = StratifiedKFold(5, shuffle = True, random_state=SEED)
inner_cv = StratifiedKFold(5, shuffle = True, random_state=SEED)
outer_cv = StratifiedKFold(5, shuffle = True, random_state=SEED)

In [391]:
y = df['label']
X = df.drop(columns = 'label')
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify = y, test_size=0.15,random_state=SEED)

In [392]:
dict_clf = { 'C': [2,7.5,8.0,9.0,10.0,12.0] ,
             'penalty': ['l2','l1'],
             'fit_intercept': [True, False]
}
clf = GridSearchCV(estimator = LogisticRegression(max_iter=1000, random_state=SEED),
                        param_grid = dict_clf,
                        cv = cv,
                        n_jobs=-1
                        )

clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test), clf.best_estimator_

ValueError: ignored

In [390]:
cross_val_score(clf.best_estimator_,  X=X, y=y, cv=outer_cv).mean()

0.5821647294088239

In [355]:
dict_clf = { 'max_depth': [8,15,12,20] ,
             'learning_rate': [0.1,1,2.0],
            'n_estimators': [5,10,100,150],
            'reg_lambda':[10,100,150,200]
}
clf = GridSearchCV(estimator = xgb.XGBClassifier(n_jobs=-1, random_state=SEED),
                        param_grid = dict_clf,
                        cv = cv,
                        n_jobs=-1
                        )

clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test), clf.best_estimator_

(0.8361266294227188,
 0.5894736842105263,
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=1,
               max_delta_step=0, max_depth=8, min_child_weight=1, missing=None,
               n_estimators=10, n_jobs=-1, nthread=None,
               objective='binary:logistic', random_state=1729, reg_alpha=0,
               reg_lambda=100, scale_pos_weight=1, seed=None, silent=None,
               subsample=1, verbosity=1))

In [356]:
 cross_val_score(clf.best_estimator_, X=X, y=y, cv=outer_cv).mean()

0.6264966879140108

In [357]:
# nested cross-validation
clf = GridSearchCV(estimator = xgb.XGBClassifier(n_jobs=-1, random_state=SEED),
                   param_grid = dict_clf,cv = inner_cv,n_jobs=-1
                    )

cross_val_score(estimator =clf, X=X, y=y, cv=outer_cv).mean()

0.6059367579052618

## Extra Trees

In [251]:
dict_clf = { 'max_depth': [10,12,18] ,
            'criterion': ['gini'],
            'n_estimators': [100,150,200,250],
             'min_samples_split': [3,5,8],
            'min_samples_leaf': [1,3],
            'max_features': ['sqrt','auto', 0.3]
}
clf = GridSearchCV(estimator = ExtraTreesClassifier(n_jobs=-1, random_state= SEED),
                        param_grid = dict_clf,
                        cv = cv,
                        n_jobs=-1
                        )

clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test), clf.best_estimator_ 

(0.9832402234636871,
 0.5263157894736842,
 ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                      criterion='gini', max_depth=12, max_features=0.3,
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
                      oob_score=False, random_state=1729, verbose=0,
                      warm_start=False))

In [252]:
 cross_val_score(ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                      criterion='entropy', max_depth=10, max_features='sqrt',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=1729, verbose=0,
                      warm_start=False),  
               X=X, y=y, cv=outer_cv, scoring='accuracy').mean()

0.6264591926009249

In [183]:
# nested cross-validation
clf = GridSearchCV(estimator = ExtraTreesClassifier(n_jobs=-1, random_state= SEED),
                   param_grid = dict_clf,cv = inner_cv,n_jobs=-1
                    )

cross_val_score(estimator =clf, X=X, y=y, cv=outer_cv).mean()

KeyboardInterrupt: ignored

## Random Forest

In [253]:
dict_clf = { 'max_depth': [10] ,
            'n_estimators': [100,150,200,250],
            'min_samples_split': [2,3,4,5],
            'min_samples_leaf': [4,5,6]
            
}
clf = GridSearchCV(estimator = RandomForestClassifier(n_jobs=-1, random_state= SEED),
                        param_grid = dict_clf,
                        cv = cv,
                        n_jobs=-1
                        )

clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test), clf.best_estimator_ 

(0.9124767225325885,
 0.5368421052631579,
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=10, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=6, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=200,
                        n_jobs=-1, oob_score=False, random_state=1729, verbose=0,
                        warm_start=False))

In [254]:
 cross_val_score(RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=10, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=4, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=150,
                        n_jobs=-1, oob_score=False, random_state=1729, verbose=0,
                        warm_start=False),  
               X=X, y=y, cv=outer_cv, scoring='accuracy').mean()

0.6074865641794776

## SVM

In [255]:
dict_clf = { 'C': [0.01,0.1,1,10,100] ,
            'degree': [1,5,10],
            'kernel': ['rbf','linear','poly','sigmoid']
            
}
clf = GridSearchCV(estimator = SVC (random_state= SEED),
                        param_grid = dict_clf,
                        cv = cv,
                        n_jobs=-1
                        )

clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test), clf.best_estimator_ 

(0.6163873370577281,
 0.5368421052631579,
 SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=1, gamma='scale', kernel='linear',
     max_iter=-1, probability=False, random_state=1729, shrinking=True,
     tol=0.001, verbose=False))

In [256]:
 cross_val_score(SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
                    decision_function_shape='ovr', degree=1, gamma='scale', kernel='linear',
                    max_iter=-1, probability=False, random_state=1729, shrinking=True,
                    tol=0.001, verbose=False),  
               X=X, y=y, cv=outer_cv, scoring='accuracy').mean()

0.5997000374953132

## Naive Bayes

In [257]:
clf = GaussianNB()
#cross_val_score(clf, X,y).mean()
clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test)

(0.5605214152700186, 0.5894736842105263)

## K Nearest Neighbors

In [258]:
dict_clf = { 'n_neighbors': [2,3,4,5,6] ,
            'weights': ['uniform', 'distance'],
            'p': [1,2,3,4,5,10],
            'leaf_size': [2,5,10,20]         
}
clf = GridSearchCV(estimator = KNeighborsClassifier(n_jobs=-1),
                        param_grid = dict_clf,
                        cv = cv,
                        n_jobs=-1
                        )

clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test), clf.best_estimator_ 

(0.6741154562383612,
 0.47368421052631576,
 KNeighborsClassifier(algorithm='auto', leaf_size=2, metric='minkowski',
                      metric_params=None, n_jobs=-1, n_neighbors=6, p=1,
                      weights='uniform'))

In [264]:
 cross_val_score(KNeighborsClassifier(algorithm='auto', leaf_size=2, metric='minkowski',
                      metric_params=None, n_jobs=-1, n_neighbors=6, p=1,
                      weights='uniform'),  
               X=X, y=y, cv=outer_cv, scoring='accuracy').mean()

0.510973628296463

## Adaboost

In [273]:
dict_clf = { 'n_estimators': [50,60,70,100],
            'learning_rate': [0.01,0.1,0.2,10],
            'base_estimator': [tree.DecisionTreeClassifier(max_depth=1, max_features= 0.75),
                               tree.DecisionTreeClassifier(max_depth=1, max_features= 0.80),
                               tree.DecisionTreeClassifier(max_depth=1, max_features= 0.9),
                               tree.DecisionTreeClassifier(max_depth=1, max_features= 0.95),
                               tree.DecisionTreeClassifier(max_depth=1)]       
}
clf = GridSearchCV(estimator = AdaBoostClassifier(random_state=SEED),
                        param_grid = dict_clf,
                        cv = cv,
                        n_jobs=-1
                        )

clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test), clf.best_estimator_ 



(0.7113594040968343,
 0.5894736842105263,
 AdaBoostClassifier(algorithm='SAMME.R',
                    base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=1,
                                                          max_features=0.8,
                                                          max_leaf_nodes=None,
                                                          min_impurity_decrease=0.0,
                                                          min_impurity_split=None,
                                                          min_samples_leaf=1,
                                                          min_samples_split=2,
                                                          min_weight_fraction_leaf=0.0,
                                                    

In [286]:
 cross_val_score(AdaBoostClassifier(algorithm='SAMME.R',
                    base_estimator=tree.DecisionTreeClassifier(ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=1,
                                                          max_features=0.8,
                                                          max_leaf_nodes=None,
                                                          min_impurity_decrease=0.0,
                                                          min_impurity_split=None,
                                                          min_samples_leaf=1,
                                                          min_samples_split=2,
                                                          min_weight_fraction_leaf=0.0,
                                                          presort='deprecated',
                                                          random_state=None,
                                                          splitter='best'),
                    learning_rate=0.1, n_estimators=50, random_state=1729),  
               X=X, y=y, cv=outer_cv, scoring='accuracy').mean()

0.6217472815898013

## Voting Classifier

### Without GridSearch

In [359]:
estimators = [('logistic', LogisticRegression( C =8.0 ,max_iter=1000 , random_state=SEED)),
               ('knn',KNeighborsClassifier(algorithm='auto', leaf_size=2, metric='minkowski',
                      metric_params=None, n_jobs=-1, n_neighbors=6, p=4,
                      weights='uniform')),
               ('svm',SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
                    decision_function_shape='ovr', degree=1, gamma='scale', kernel='linear',
                    max_iter=-1, probability=False, random_state=1729, shrinking=True,
                    tol=0.001, verbose=False)),
               ('xgb', xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0,
               learning_rate=2.0, max_delta_step=0, max_depth=8,
               min_child_weight=1, missing=None, n_estimators=5, n_jobs=-1,
               nthread=None, objective='binary:logistic', random_state=1729,
               reg_alpha=0, reg_lambda=100, scale_pos_weight=1, seed=None,
               silent=None, subsample=1, verbosity=1)),
              ('nb',GaussianNB()),
              ('rf', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=10, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=4, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=150,
                        n_jobs=-1, oob_score=False, random_state=1729, verbose=0,
                        warm_start=False))
               ]

clf = VotingClassifier(estimators=estimators, voting='hard')
clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test),  cross_val_score(VotingClassifier(estimators=estimators), X=X, y=y, cv=outer_cv, scoring='accuracy').mean()

(0.7299813780260708, 0.5578947368421052, 0.6154230721159856)

In [362]:
estimators = [('logistic', LogisticRegression( C =8.0 ,max_iter=1000 , random_state=SEED)),
               ('rf', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=10, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=4, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=150,
                        n_jobs=-1, oob_score=False, random_state=1729, verbose=0,
                        warm_start=False)),
              ('nb',GaussianNB())
               ]

clf = VotingClassifier(estimators=estimators, voting='hard')
clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test),  cross_val_score(VotingClassifier(estimators=estimators), X=X, y=y, cv=outer_cv, scoring='accuracy').mean()

(0.7150837988826816, 0.631578947368421, 0.6074990626171729)

In [303]:
estimators = [('logistic', LogisticRegression( C =8.0 ,max_iter=1000 , random_state=SEED)),
               
              ('nb',GaussianNB()),
              ('et', ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                      criterion='entropy', max_depth=10, max_features='sqrt',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=1729, verbose=0,
                      warm_start=False))
               ]

clf = VotingClassifier(estimators=estimators)
clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test),  cross_val_score(VotingClassifier(estimators=estimators), X=X, y=y, cv=outer_cv, scoring='accuracy').mean()

(0.7132216014897579, 0.631578947368421, 0.6154230721159856)

## Stacking Classifier

In [364]:
estimators = [('logistic', LogisticRegression( C =8.0 ,max_iter=1000 , random_state=SEED)),
               ('knn',KNeighborsClassifier(algorithm='auto', leaf_size=2, metric='minkowski',
                      metric_params=None, n_jobs=-1, n_neighbors=6, p=4,
                      weights='uniform')),
               ('svm',SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
                    decision_function_shape='ovr', degree=1, gamma='scale', kernel='linear',
                    max_iter=-1, probability=False, random_state=1729, shrinking=True,
                    tol=0.001, verbose=False)),
              ('nb',GaussianNB()),
               ('xgb', xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0,
               learning_rate=2.0, max_delta_step=0, max_depth=8,
               min_child_weight=1, missing=None, n_estimators=5, n_jobs=-1,
               nthread=None, objective='binary:logistic', random_state=1729,
               reg_alpha=0, reg_lambda=100, scale_pos_weight=1, seed=None,
               silent=None, subsample=1, verbosity=1)),
              ('rf', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=10, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=4, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=150,
                        n_jobs=-1, oob_score=False, random_state=1729, verbose=0,
                        warm_start=False))
               ]

clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test),  cross_val_score(clf, X=X, y=y, cv=outer_cv, scoring='accuracy').mean()


(0.8640595903165735, 0.5684210526315789, 0.6027621547306585)

In [365]:
estimators = [
               ('rf', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=10, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=4, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=150,
                        n_jobs=-1, oob_score=False, random_state=1729,
                        warm_start=False)),
              ('nb',GaussianNB())
               ]

clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), stack_method='predict')
clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test),  cross_val_score(clf, X=X, y=y, cv=outer_cv, scoring='accuracy').mean()

(0.9497206703910615, 0.6, 0.6074865641794776)

In [366]:
estimators = [('logistic', LogisticRegression( C =8.0 ,max_iter=1000 , random_state=SEED)),
               
              ('nb',GaussianNB()),
              ('et', ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                      criterion='entropy', max_depth=10, max_features='sqrt',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=1729, verbose=0,
                      warm_start=False))
               ]

clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), stack_method='predict')
clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test),  cross_val_score(clf, X=X, y=y, cv=outer_cv, scoring='accuracy').mean()

(0.8659217877094972, 0.5894736842105263, 0.6217472815898013)

In [280]:
dict_classifiers = {
    'Extra Trees': ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                      criterion='entropy', max_depth=10, max_features='sqrt',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=1729, verbose=0,
                      warm_start=False),
                    
    "Logistic Regression": LogisticRegression( C =8.0 ,max_iter=1000 , random_state=SEED),

    "Nearest Neighbors": KNeighborsClassifier(algorithm='auto', leaf_size=2, metric='minkowski',
                      metric_params=None, n_jobs=-1, n_neighbors=6, p=4,
                      weights='uniform'),
                    
    "SVM": SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
                    decision_function_shape='ovr', degree=1, gamma='scale', kernel='linear',
                    max_iter=-1, probability=False, random_state=1729, shrinking=True,
                    tol=0.001, verbose=False),

    "Gradient Boosting Classifier": GradientBoostingClassifier(n_estimators=1000),
    "Decision Tree": tree.DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=10, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=4, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=150,
                        n_jobs=-1, oob_score=False, random_state=1729, verbose=0,
                        warm_start=False),
    "Neural Net": MLPClassifier(alpha = 1),
    "Naive Bayes": GaussianNB(),
    "AdaBoost": AdaBoostClassifier(algorithm='SAMME.R',
                    base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=1,
                                                          max_features=0.8,
                                                          max_leaf_nodes=None,
                                                          min_impurity_decrease=0.0,
                                                          min_impurity_split=None,
                                                          min_samples_leaf=1,
                                                          min_samples_split=2,
                                                          min_weight_fraction_leaf=0.0,
                                                          presort='deprecated',
                                                          random_state=None,
                                                          splitter='best'),
                    learning_rate=0.1, n_estimators=50, random_state=1729),
    "QDA": QuadraticDiscriminantAnalysis(),
    "Gaussian Process": GaussianProcessClassifier(),
    "XGBoost": xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0,
               learning_rate=2.0, max_delta_step=0, max_depth=8,
               min_child_weight=1, missing=None, n_estimators=5, n_jobs=-1,
               nthread=None, objective='binary:logistic', random_state=1729,
               reg_alpha=0, reg_lambda=100, scale_pos_weight=1, seed=None,
               silent=None, subsample=1, verbosity=1)
}

In [281]:
def batch_classify(X_train, Y_train, X_test, Y_test, no_classifiers = 5, verbose = True):
    
    dict_models = {}
    for classifier_name, classifier in list(dict_classifiers.items())[:no_classifiers]:
        t_start = time.clock()
        classifier.fit(X_train, Y_train)
        t_end = time.clock()
        
        t_diff = t_end - t_start
        train_score = classifier.score(X_train, Y_train)
        test_score = classifier.score(X_test, Y_test)
        
        dict_models[classifier_name] = {'model': classifier, 'train_score': train_score, 'test_score': test_score, 'train_time': t_diff}
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=classifier_name, f=t_diff))
    return dict_models



def display_dict_models(dict_models, sort_by='test_score'):
    cls = [key for key in dict_models.keys()]
    test_s = [dict_models[key]['test_score'] for key in cls]
    training_s = [dict_models[key]['train_score'] for key in cls]
    training_t = [dict_models[key]['train_time'] for key in cls]
    
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),4)), columns = ['classifier', 'train_score', 'test_score', 'train_time'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'train_score'] = training_s[ii]
        df_.loc[ii, 'test_score'] = test_s[ii]
        df_.loc[ii, 'train_time'] = training_t[ii]
    
    display(df_.sort_values(by=sort_by, ascending=False))

In [282]:
dict_models = batch_classify(X_train, y_train, X_test, y_test, no_classifiers = 13)
display_dict_models(dict_models)

trained Extra Trees in 0.22 s
trained Logistic Regression in 0.33 s
trained Nearest Neighbors in 0.01 s
trained SVM in 2.22 s
trained Gradient Boosting Classifier in 3.16 s
trained Decision Tree in 0.01 s
trained Random Forest in 0.52 s
trained Neural Net in 0.63 s
trained Naive Bayes in 0.00 s
trained AdaBoost in 0.26 s
trained QDA in 0.00 s


/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


trained Gaussian Process in 0.23 s
trained XGBoost in 0.05 s


,classifier,train_score,test_score,train_time
1,Logistic Regression,0.648045,0.621053,0.325968
6,Random Forest,0.949721,0.600000,0.517783
0,Extra Trees,0.923650,0.589474,0.220691
8,Naive Bayes,0.560521,0.589474,0.003736
9,AdaBoost,0.711359,0.589474,0.256435
12,XGBoost,0.813780,0.578947,0.048956
5,Decision Tree,1.000000,0.547368,0.008930
11,Gaussian Process,1.000000,0.547368,0.227960
3,SVM,0.616387,0.536842,2.215376
2,Nearest Neighbors,0.662942,0.526316,0.009835
